# SAE Project - Session 3: Quality Inspection and Control

## Session Objectives (4 hours)

1. Implement sensor-based quality inspection simulation
2. Develop quality classification algorithm
3. Use Jacobian for velocity control during inspection
4. Integrate feedback control for precise positioning

---

## Part 3.1: Vision System Simulation

### Background

In Industry 4.0, quality inspection often uses vision systems with various sensors:
- **Cameras**: Dimensional measurement, defect detection
- **Laser scanners**: 3D profiling
- **Sensors**: Position, orientation verification

For this project, we'll simulate a vision inspection system that checks:
1. **Dimensional accuracy**: Component size within tolerance
2. **Position accuracy**: Component properly positioned
3. **Defect detection**: Surface defects, missing features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from roboticstoolbox import DHRobot, RevoluteDH
from spatialmath import SE3
import roboticstoolbox as rtb
from dataclasses import dataclass
from enum import Enum
import random

%matplotlib inline

## Task 3.1: Simulated Component Generator

Create a generator that produces components with varying quality levels

In [ ]:
# Task 3.1: Component quality simulator

class QualityLevel(Enum):
    """Quality classification levels"""
    GOOD = "GOOD"          # Perfect component
    REWORK = "REWORK"      # Minor defects, repairable
    REJECT = "REJECT"      # Major defects, scrap

@dataclass
class Component:
    """Electronic component with quality parameters"""
    id: int
    position: tuple  # Tray position
    
    # Dimensional parameters (mm)
    length: float = 20.0
    width: float = 20.0
    height: float = 10.0
    
    # Position errors (mm)
    pos_error_x: float = 0.0
    pos_error_y: float = 0.0
    rotation_error: float = 0.0  # degrees
    
    # Defects
    has_crack: bool = False
    has_scratch: bool = False
    missing_component: bool = False
    
    # Quality level (ground truth)
    quality: QualityLevel = QualityLevel.GOOD

class ComponentGenerator:
    """
    Generate components with realistic quality distributions
    
    Typical Industry 4.0 quality distribution:
    - 85% GOOD
    - 10% REWORK (minor defects)
    - 5% REJECT (major defects)
    """
    def __init__(self, seed=42):
        random.seed(seed)
        np.random.seed(seed)
        
        # Quality distribution
        self.quality_probs = {
            QualityLevel.GOOD: 0.85,
            QualityLevel.REWORK: 0.10,
            QualityLevel.REJECT: 0.05
        }
        
        # Tolerance specifications (mm)
        self.nominal_size = [20.0, 20.0, 10.0]  # L, W, H
        self.size_tolerance = 0.2  # ±0.2mm
        self.position_tolerance = 1.0  # ±1.0mm
        self.rotation_tolerance = 2.0  # ±2 degrees
    
    def generate_component(self, component_id, tray_position):
        """
        Generate a component with random quality characteristics
        """
        # Randomly select quality level
        quality = np.random.choice(
            list(self.quality_probs.keys()),
            p=list(self.quality_probs.values())
        )
        
        # Generate dimensions based on quality
        if quality == QualityLevel.GOOD:
            # Tight tolerances, no defects
            length = self.nominal_size[0] + np.random.uniform(-0.05, 0.05)
            width = self.nominal_size[1] + np.random.uniform(-0.05, 0.05)
            height = self.nominal_size[2] + np.random.uniform(-0.05, 0.05)
            pos_error_x = np.random.uniform(-0.2, 0.2)
            pos_error_y = np.random.uniform(-0.2, 0.2)
            rotation_error = np.random.uniform(-0.5, 0.5)
            has_crack = False
            has_scratch = False
            missing_component = False
            
        elif quality == QualityLevel.REWORK:
            # Moderate dimensional errors or minor defects
            length = self.nominal_size[0] + np.random.uniform(-0.15, 0.15)
            width = self.nominal_size[1] + np.random.uniform(-0.15, 0.15)
            height = self.nominal_size[2] + np.random.uniform(-0.15, 0.15)
            pos_error_x = np.random.uniform(-0.8, 0.8)
            pos_error_y = np.random.uniform(-0.8, 0.8)
            rotation_error = np.random.uniform(-1.5, 1.5)
            has_crack = False
            has_scratch = random.random() < 0.6  # 60% chance
            missing_component = False
            
        else:  # REJECT
            # Large dimensional errors or major defects
            length = self.nominal_size[0] + np.random.uniform(-0.5, 0.5)
            width = self.nominal_size[1] + np.random.uniform(-0.5, 0.5)
            height = self.nominal_size[2] + np.random.uniform(-0.3, 0.3)
            pos_error_x = np.random.uniform(-2.0, 2.0)
            pos_error_y = np.random.uniform(-2.0, 2.0)
            rotation_error = np.random.uniform(-5.0, 5.0)
            has_crack = random.random() < 0.7  # 70% chance
            has_scratch = random.random() < 0.5
            missing_component = random.random() < 0.3
        
        return Component(
            id=component_id,
            position=tray_position,
            length=length,
            width=width,
            height=height,
            pos_error_x=pos_error_x,
            pos_error_y=pos_error_y,
            rotation_error=rotation_error,
            has_crack=has_crack,
            has_scratch=has_scratch,
            missing_component=missing_component,
            quality=quality
        )
    
    def generate_batch(self, n_components=9):
        """
        Generate a batch of components
        """
        components = []
        for i in range(n_components):
            row = i // 3
            col = i % 3
            comp = self.generate_component(i, (row, col))
            components.append(comp)
        return components

# Test component generator
generator = ComponentGenerator()
test_batch = generator.generate_batch(9)

print("Sample component batch:")
print("ID | Position | Quality  | Dimensions | Defects")
print("-" * 70)
for comp in test_batch:
    defects = []
    if comp.has_crack: defects.append("crack")
    if comp.has_scratch: defects.append("scratch")
    if comp.missing_component: defects.append("missing part")
    defect_str = ", ".join(defects) if defects else "none"
    
    print(f"{comp.id:2d} | {comp.position} | {comp.quality.value:8s} | "
          f"{comp.length:.2f}x{comp.width:.2f}x{comp.height:.2f} | {defect_str}")

## Task 3.2: Vision Inspection Algorithm

**Objective:** Implement inspection algorithm that classifies components based on sensor data

**Inspection Criteria:**
- **GOOD**: All dimensions within ±0.2mm, no defects
- **REWORK**: Dimensions within ±0.3mm, minor scratches acceptable
- **REJECT**: Dimensions outside ±0.3mm, or cracks/missing parts

In [ ]:
# Task 3.2: Vision inspection system

class VisionInspectionSystem:
    """
    Simulates a vision-based quality inspection system
    """
    def __init__(self):
        # Inspection thresholds
        self.size_tolerance_good = 0.2  # mm
        self.size_tolerance_rework = 0.3  # mm
        self.nominal_size = [20.0, 20.0, 10.0]
        
        # Measurement noise (simulates sensor uncertainty)
        self.measurement_noise = 0.01  # mm
    
    def measure_dimensions(self, component):
        """
        Measure component dimensions with simulated sensor noise
        """
        length = component.length + np.random.normal(0, self.measurement_noise)
        width = component.width + np.random.normal(0, self.measurement_noise)
        height = component.height + np.random.normal(0, self.measurement_noise)
        
        return np.array([length, width, height])
    
    def detect_defects(self, component):
        """
        Detect surface defects
        Returns dict with defect probabilities (simulates AI/CV system)
        """
        # Simulate detection with some false positives/negatives
        crack_prob = 0.95 if component.has_crack else 0.02
        scratch_prob = 0.90 if component.has_scratch else 0.05
        missing_prob = 0.98 if component.missing_component else 0.01
        
        detected_crack = np.random.random() < crack_prob
        detected_scratch = np.random.random() < scratch_prob
        detected_missing = np.random.random() < missing_prob
        
        return {
            'crack': detected_crack,
            'scratch': detected_scratch,
            'missing_component': detected_missing
        }
    
    def inspect_component(self, component):
        """
        Complete inspection: measurements + defect detection
        
        Returns:
            quality_decision: QualityLevel
            inspection_report: Dict with details
        """
        # Measure dimensions
        measured_dims = self.measure_dimensions(component)
        dim_errors = np.abs(measured_dims - self.nominal_size)
        max_dim_error = np.max(dim_errors)
        
        # Detect defects
        defects = self.detect_defects(component)
        
        # Decision logic
        quality_decision = self._classify_quality(max_dim_error, defects)
        
        # Inspection report
        report = {
            'component_id': component.id,
            'measured_dims': measured_dims,
            'dim_errors': dim_errors,
            'max_error': max_dim_error,
            'defects': defects,
            'decision': quality_decision,
            'ground_truth': component.quality
        }
        
        return quality_decision, report
    
    def _classify_quality(self, max_dim_error, defects):
        """
        Classification logic based on inspection results
        
        TODO: Implement your classification algorithm
        """
        # Major defects → REJECT
        if defects['crack'] or defects['missing_component']:
            return QualityLevel.REJECT
        
        # Large dimensional errors → REJECT
        if max_dim_error > self.size_tolerance_rework:
            return QualityLevel.REJECT
        
        # Minor defects or small dimensional errors → REWORK
        if defects['scratch'] or max_dim_error > self.size_tolerance_good:
            return QualityLevel.REWORK
        
        # Perfect → GOOD
        return QualityLevel.GOOD

# Test inspection system
vision_system = VisionInspectionSystem()

print("\nInspection Results:")
print("ID | Decision | Ground Truth | Max Error | Defects")
print("-" * 80)

correct = 0
for comp in test_batch:
    decision, report = vision_system.inspect_component(comp)
    
    defect_list = [k for k, v in report['defects'].items() if v]
    defect_str = ", ".join(defect_list) if defect_list else "none"
    
    match = "✓" if decision == comp.quality else "✗"
    if decision == comp.quality:
        correct += 1
    
    print(f"{comp.id:2d} | {decision.value:8s} | {comp.quality.value:12s} | "
          f"{report['max_error']:.3f} mm | {defect_str:20s} {match}")

accuracy = 100 * correct / len(test_batch)
print(f"\nInspection Accuracy: {accuracy:.1f}%")

## Task 3.3: Jacobian-Based Velocity Control

**Objective:** Use the Jacobian for smooth velocity control during vision scanning

**Background:**
- Vision systems often require constant velocity for optimal image quality
- Jacobian relates joint velocities to end-effector velocities: $\mathbf{v} = \mathbf{J}(\mathbf{q}) \dot{\mathbf{q}}$
- We can control Cartesian velocity by computing required joint velocities

In [ ]:
# Task 3.3: Jacobian-based velocity control

# Load robot model
class OpenManipulatorX(DHRobot):
    def __init__(self):
        d1 = 0.077
        a2 = 0.130
        a3 = 0.124
        a4 = 0.126
        
        L = [
            RevoluteDH(d=d1, a=0, alpha=np.pi/2),
            RevoluteDH(d=0, a=a2, alpha=0),
            RevoluteDH(d=0, a=a3, alpha=0),
            RevoluteDH(d=0, a=a4, alpha=0),
        ]
        
        super().__init__(L, name="OpenManipulator-X")

robot = OpenManipulatorX()

def velocity_control_jacobian(robot, q, desired_velocity):
    """
    Compute joint velocities to achieve desired end-effector velocity
    
    Args:
        robot: Robot model
        q: Current joint configuration
        desired_velocity: Desired end-effector velocity [vx, vy, vz, wx, wy, wz]
    
    Returns:
        qd: Joint velocities
    """
    # Compute Jacobian at current configuration
    J = robot.jacob0(q)
    
    # For 4-DOF robot, we have limited DOF
    # Typically use pseudo-inverse for redundancy resolution
    # TODO: Implement Jacobian inverse/pseudo-inverse
    
    # Method 1: Pseudo-inverse (Moore-Penrose)
    J_pinv = np.linalg.pinv(J)
    qd = J_pinv @ desired_velocity
    
    return qd

# Test velocity control for vision scanning
# Desired: Move in +X direction at constant 20 mm/s
q_test = [0, np.pi/6, -np.pi/4, np.pi/12]
desired_vel = np.array([0.02, 0, 0, 0, 0, 0])  # 20 mm/s in X, no rotation

qd = velocity_control_jacobian(robot, q_test, desired_vel)

print("Velocity Control Test:")
print(f"Desired end-effector velocity: {desired_vel[:3] * 1000} mm/s")
print(f"Required joint velocities: {np.rad2deg(qd)} deg/s")

# Verify
J = robot.jacob0(q_test)
achieved_vel = J @ qd
print(f"Achieved end-effector velocity: {achieved_vel[:3] * 1000} mm/s")
print(f"Error: {np.linalg.norm(achieved_vel[:3] - desired_vel[:3]) * 1000:.6f} mm/s")

## Task 3.4: Constant Velocity Trajectory for Vision Scanning

Generate a trajectory that maintains constant end-effector velocity during scanning

In [ ]:
# Task 3.4: Constant velocity scanning trajectory

def generate_constant_velocity_scan(robot, p_start, p_end, velocity=0.02, dt=0.01):
    """
    Generate trajectory with constant Cartesian velocity
    
    Args:
        robot: Robot model
        p_start: Start position [x, y, z]
        p_end: End position [x, y, z]
        velocity: Desired speed (m/s)
        dt: Time step
    
    Returns:
        t, Q, Qd: Time, joint positions, joint velocities
    """
    # Calculate scan distance and duration
    distance = np.linalg.norm(np.array(p_end) - np.array(p_start))
    duration = distance / velocity
    
    # Direction vector
    direction = (np.array(p_end) - np.array(p_start)) / distance
    
    # Time vector
    t = np.arange(0, duration + dt, dt)
    n_steps = len(t)
    
    # Initialize trajectories
    Q = np.zeros((n_steps, robot.n))
    Qd = np.zeros((n_steps, robot.n))
    P = np.zeros((n_steps, 3))
    
    # Initial configuration
    from scipy.optimize import fsolve
    # Simple IK for initial position
    Tep = SE3(p_start[0], p_start[1], p_start[2]) @ SE3.RPY([0, np.pi, 0])
    sol = robot.ikine_LM(Tep)
    q_current = sol.q if sol.success else np.zeros(robot.n)
    
    # Generate trajectory
    for i in range(n_steps):
        # Current position along line
        s = velocity * t[i]
        p_current = np.array(p_start) + direction * s
        P[i, :] = p_current
        
        # Desired velocity (constant along direction)
        v_desired = np.concatenate([direction * velocity, [0, 0, 0]])
        
        # Compute joint velocities using Jacobian
        qd_desired = velocity_control_jacobian(robot, q_current, v_desired)
        
        # Update joint positions (forward Euler integration)
        if i > 0:
            q_current = q_current + qd_desired * dt
        
        Q[i, :] = q_current
        Qd[i, :] = qd_desired
    
    return t, Q, Qd, P

# Test constant velocity scanning
scan_start = [0.000, 0.150, 0.080]
scan_end = [0.060, 0.150, 0.080]
scan_velocity = 0.030  # 30 mm/s

t_scan, Q_scan, Qd_scan, P_scan = generate_constant_velocity_scan(
    robot, scan_start, scan_end, velocity=scan_velocity
)

# Analyze velocity profile
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Cartesian position
axes[0, 0].plot(t_scan, P_scan[:, 0] * 1000, label='X')
axes[0, 0].plot(t_scan, P_scan[:, 1] * 1000, label='Y')
axes[0, 0].plot(t_scan, P_scan[:, 2] * 1000, label='Z')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Position (mm)')
axes[0, 0].set_title('Cartesian Position')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Cartesian velocity magnitude
if len(P_scan) > 1:
    cart_vel = np.diff(P_scan, axis=0) / np.diff(t_scan)[:, np.newaxis]
    cart_speed = np.linalg.norm(cart_vel, axis=1)
    axes[0, 1].plot(t_scan[:-1], cart_speed * 1000)
    axes[0, 1].axhline(scan_velocity * 1000, color='r', linestyle='--', 
                       label=f'Target: {scan_velocity*1000:.0f} mm/s')
    axes[0, 1].set_xlabel('Time (s)')
    axes[0, 1].set_ylabel('Speed (mm/s)')
    axes[0, 1].set_title('End-Effector Speed')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

# Joint positions
for i in range(robot.n):
    axes[1, 0].plot(t_scan, np.rad2deg(Q_scan[:, i]), label=f'Joint {i+1}')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Position (deg)')
axes[1, 0].set_title('Joint Positions')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Joint velocities
for i in range(robot.n):
    axes[1, 1].plot(t_scan, np.rad2deg(Qd_scan[:, i]))
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Velocity (deg/s)')
axes[1, 1].set_title('Joint Velocities')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"Scan duration: {t_scan[-1]:.2f} s")
print(f"Scan distance: {np.linalg.norm(np.array(scan_end) - np.array(scan_start)) * 1000:.1f} mm")
if len(P_scan) > 1:
    print(f"Average speed: {np.mean(cart_speed) * 1000:.1f} mm/s")
    print(f"Speed std dev: {np.std(cart_speed) * 1000:.3f} mm/s")

## Task 3.5: Integration - Complete Inspection Workflow

Combine motion planning with quality inspection

In [ ]:
# Task 3.5: Integrated quality control system

class QualityControlCell:
    """
    Complete quality control cell integrating:
    - Motion planning
    - Vision inspection
    - Sorting logic
    """
    def __init__(self, robot, vision_system, component_generator):
        self.robot = robot
        self.vision = vision_system
        self.generator = component_generator
        
        # Statistics
        self.stats = {
            'total_processed': 0,
            'good_count': 0,
            'rework_count': 0,
            'reject_count': 0,
            'correct_classifications': 0,
            'total_time': 0
        }
    
    def process_component(self, component):
        """
        Process one component through the complete workflow
        
        Returns:
            result: Dict with processing results
        """
        start_time = 0  # Would be actual time in real system
        
        # Step 1: Inspect component
        quality_decision, inspection_report = self.vision.inspect_component(component)
        
        # Step 2: Determine target bin
        if quality_decision == QualityLevel.GOOD:
            bin_type = 'GOOD'
            self.stats['good_count'] += 1
        elif quality_decision == QualityLevel.REWORK:
            bin_type = 'REWORK'
            self.stats['rework_count'] += 1
        else:
            bin_type = 'REJECT'
            self.stats['reject_count'] += 1
        
        # Step 3: Update statistics
        self.stats['total_processed'] += 1
        if quality_decision == component.quality:
            self.stats['correct_classifications'] += 1
        
        # Prepare result
        result = {
            'component_id': component.id,
            'ground_truth': component.quality.value,
            'decision': quality_decision.value,
            'bin': bin_type,
            'correct': quality_decision == component.quality,
            'inspection_report': inspection_report
        }
        
        return result
    
    def process_batch(self, components):
        """
        Process a batch of components
        """
        results = []
        for comp in components:
            result = self.process_component(comp)
            results.append(result)
        
        return results
    
    def print_statistics(self):
        """
        Print quality control statistics
        """
        print("\n" + "="*60)
        print("QUALITY CONTROL STATISTICS")
        print("="*60)
        print(f"Total components processed: {self.stats['total_processed']}")
        print(f"  → GOOD bin:   {self.stats['good_count']:3d} "
              f"({100*self.stats['good_count']/max(1,self.stats['total_processed']):.1f}%)")
        print(f"  → REWORK bin: {self.stats['rework_count']:3d} "
              f"({100*self.stats['rework_count']/max(1,self.stats['total_processed']):.1f}%)")
        print(f"  → REJECT bin: {self.stats['reject_count']:3d} "
              f"({100*self.stats['reject_count']/max(1,self.stats['total_processed']):.1f}%)")
        print(f"\nClassification accuracy: "
              f"{100*self.stats['correct_classifications']/max(1,self.stats['total_processed']):.1f}%")
        print("="*60)

# Create and test quality control cell
qc_cell = QualityControlCell(robot, vision_system, generator)

# Generate and process a production batch
production_batch = generator.generate_batch(50)  # 50 components
results = qc_cell.process_batch(production_batch)

# Display results
qc_cell.print_statistics()

# Confusion matrix
from collections import defaultdict
confusion = defaultdict(lambda: defaultdict(int))

for result in results:
    confusion[result['ground_truth']][result['decision']] += 1

print("\nConfusion Matrix:")
print("" + ""*50)
print(f"{'':12s} | {'Classified as →':^40s}")
print(f"{'Ground Truth':12s} | {'GOOD':>12s} {'REWORK':>12s} {'REJECT':>12s}")
print("-" * 54)
for true_label in ['GOOD', 'REWORK', 'REJECT']:
    print(f"{true_label:12s} | ", end="")
    for pred_label in ['GOOD', 'REWORK', 'REJECT']:
        count = confusion[true_label][pred_label]
        print(f"{count:12d}", end=" ")
    print()

---

## Session 3 Deliverables

**By the end of Session 3, you should have:**

✅ Component quality simulator with realistic defect distributions  
✅ Vision inspection system with measurement and defect detection  
✅ Quality classification algorithm  
✅ Jacobian-based velocity control implementation  
✅ Constant velocity trajectory for vision scanning  
✅ Integrated quality control workflow  
✅ Statistical analysis of inspection performance  

**Questions for Report:**

1. How does sensor noise affect classification accuracy?
2. What is the trade-off between scan speed and inspection quality?
3. Why is constant velocity important for vision systems?
4. How would you improve the classification algorithm to reduce false positives/negatives?
5. What Industry 4.0 features could enhance this quality control system (IoT, AI, data analytics)?

---

**Continue to Session 4 notebook for final system integration and optimization...**